In [1]:
import torch
import torchvision as tv

import pandas as pd
import numpy as np

### 1. Скачайте тренировочную и тестовою часть датасета Fashion MNIST


In [2]:
BatchSize = 256

train_dataset = tv.datasets.FashionMNIST('.', train = True, transform = tv.transforms.ToTensor(), download = True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform = tv.transforms.ToTensor(), download = True)
train = torch.utils.data.DataLoader(train_dataset, batch_size = BatchSize, shuffle = True)
test = torch.utils.data.DataLoader(test_dataset, batch_size = BatchSize, shuffle = True)

train_dataset[0][0].shape

torch.Size([1, 28, 28])

### 2. Постройте модель, выбрав стартовую архитектуру


In [3]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [4]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.01)
num_epochs = 10

### 3. Обучите модель и сверьте качество на тестовой части с заданным порогом


In [5]:
import time

def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train:
            optimizer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        print("ep: {}, time: {:.3f}, train_loss: {:.6f}, train_acc: {:.6f}, test_loss: {:.6f}, test_acc: {:.6f}".format(
            ep+1, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [6]:
train_model()

ep: 1, time: 9.797, train_loss: 0.463484, train_acc: 0.828167, test_loss: 0.403498, test_acc: 0.850000
ep: 2, time: 10.022, train_loss: 0.344198, train_acc: 0.872467, test_loss: 0.361498, test_acc: 0.866500
ep: 3, time: 10.145, train_loss: 0.298877, train_acc: 0.888317, test_loss: 0.350229, test_acc: 0.872300
ep: 4, time: 10.176, train_loss: 0.274886, train_acc: 0.896333, test_loss: 0.378572, test_acc: 0.865800
ep: 5, time: 10.173, train_loss: 0.253937, train_acc: 0.903883, test_loss: 0.361604, test_acc: 0.870700
ep: 6, time: 10.199, train_loss: 0.234114, train_acc: 0.911117, test_loss: 0.349672, test_acc: 0.871000
ep: 7, time: 10.254, train_loss: 0.221179, train_acc: 0.917183, test_loss: 0.381089, test_acc: 0.869000
ep: 8, time: 10.276, train_loss: 0.206067, train_acc: 0.922050, test_loss: 0.343048, test_acc: 0.881100
ep: 9, time: 10.241, train_loss: 0.189118, train_acc: 0.928367, test_loss: 0.395982, test_acc: 0.876400
ep: 10, time: 10.184, train_loss: 0.179903, train_acc: 0.931950, 

### 4. Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.

In [21]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.Dropout(0.3),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.Dropout(0.3),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.Dropout(0.3),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [22]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.01)
num_epochs = 20

In [23]:
train_model()

ep: 1, time: 12.794, train_loss: 0.513673, train_acc: 0.815467, test_loss: 0.449510, test_acc: 0.835900
ep: 2, time: 12.763, train_loss: 0.396838, train_acc: 0.856750, test_loss: 0.402423, test_acc: 0.855300
ep: 3, time: 12.781, train_loss: 0.352971, train_acc: 0.870817, test_loss: 0.378719, test_acc: 0.864100
ep: 4, time: 14.282, train_loss: 0.332042, train_acc: 0.878950, test_loss: 0.348784, test_acc: 0.870300
ep: 5, time: 13.227, train_loss: 0.309932, train_acc: 0.885683, test_loss: 0.393002, test_acc: 0.857600
ep: 6, time: 12.971, train_loss: 0.297630, train_acc: 0.890000, test_loss: 0.332607, test_acc: 0.880000
ep: 7, time: 13.105, train_loss: 0.282188, train_acc: 0.895233, test_loss: 0.326595, test_acc: 0.882000
ep: 8, time: 12.976, train_loss: 0.274242, train_acc: 0.898433, test_loss: 0.332883, test_acc: 0.880100
ep: 9, time: 14.470, train_loss: 0.264783, train_acc: 0.901650, test_loss: 0.331871, test_acc: 0.882400
ep: 10, time: 15.004, train_loss: 0.254179, train_acc: 0.904300,

In [24]:
# Добавили слоев и расшири количество нейронов, также добавили слои с Дропаутом. 
# Дополнительно немного поэкспеременитровав, оешил увеличить количество эпох до 20.

"""Результат test_acc: 0.896100"""

'Результат test_acc: 0.896100'